In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
from tqdm import tqdm
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn import metrics
from scipy import stats
from itertools import product

Lang = 'Java'
#Lang = 'C'

if Lang == 'Java':
    projects = ['Lang', 'Chart', 'Time', 'Math', 'Closure']
    N = 8
else:
    projects = ['grep', 'gzip', 'sed', 'flex']
    N = 5

meta = pd.DataFrame()
result = pd.DataFrame()
fl_result = pd.DataFrame()
dc_result = pd.DataFrame()
for project in projects:
    for num_faults in range(1, N+1):
        if Lang == 'Java':
            dataset_id = f"{project}-faults-{num_faults}"
        else:
            dataset_id = f"{project}-faults-{num_faults}"
        mp, rp, flp, dcp = (
            f"resources/result/{dataset_id}_meta.pkl",
            f"resources/result/{dataset_id}_result.pkl",
            f"resources/result/{dataset_id}_fl_result.pkl",
            f"resources/result/{dataset_id}_dist_cost_result.pkl")
        if os.path.exists(mp) and os.path.exists(rp):
            t_meta = pd.read_pickle(mp)
            t_result = pd.read_pickle(rp)

            print(dataset_id, t_meta.shape[0])
            t_meta['project'] = project
            meta = meta.append(t_meta, ignore_index=True)
            result = result.append(t_result, ignore_index=True)
            if os.path.exists(flp):
                t_fl_result = pd.read_pickle(flp)
                fl_result = fl_result.append(t_fl_result, ignore_index=True)
            if os.path.exists(dcp):
                t_dc_result = pd.read_pickle(dcp)
                dc_result = dc_result.append(t_dc_result, ignore_index=True)
        else:
            print(dataset_id, "No result file")

assert meta['data_id'].unique().shape[0] == meta.shape[0]

print("# total data points:", meta.shape[0])
for k, df in meta.groupby(['num_faults']):
    print(f"{k} faults: {df.shape[0]}", 
        [ f"{project}: {df[df.project == project].shape[0]}" for project in projects])

print(np.all(result.normalized_mutual_info_score - result.v_measure_score < 0.0001))

print(meta.groupby(['project', 'num_faults'])['data_id'].nunique().reset_index().to_markdown(index=False))

affinity_name_map = {'nlink': 'hdist'}
for affinity in affinity_name_map:
    new_name = affinity_name_map[affinity]
    result.loc[result.affinity == affinity, 'affinity'] = new_name
    dc_result.loc[dc_result.affinity == affinity, 'affinity'] = new_name

# Subject Statistics

## Averaged # failing test cases for each # faults

In [ ]:
print("Total:", pd.to_numeric(meta.set_index(['data_id', 'project', 'num_faults']).num_vertices).mean())

pd.to_numeric(meta.set_index(['data_id', 'project', 'num_faults']).num_vertices).mean(level=['num_faults', 'project']).to_frame().round(2)

## Averaged # lines (components) per projects

In [ ]:
print("Total:", pd.to_numeric(meta.set_index(['data_id']).num_total_components).mean())
meta['num_total_components'] = meta['num_total_components'].astype('int32')
meta.groupby(['project'])['num_total_components'].mean().to_frame().round(1)

## the # data points for each # failing test cases

In [ ]:
print(meta[meta['num_vertices'] <= 2].shape[0]/meta.shape[0])

meta.groupby(['num_vertices']).count()['data_id'].to_frame()

# Some helper functions

In [ ]:
def filtering(result_df, filters_list):
    sdf = pd.DataFrame()
    for filters in filters_list:
        tmp_df = result_df
        method = '-'.join(filters.values())
        for column in filters:
            tmp_df = tmp_df[tmp_df[column] == filters[column]]
        tmp_df['method'] = method
        sdf = sdf.append(tmp_df, ignore_index=True)
    return sdf

In [ ]:
def stop_at_max_modularity(result_df, filters=None):
    if filters is not None:
        result_df = filtering(result_df, [filters])
    return result_df.sort_values(['modularity', 'iteration'], ascending=[False, True])\
        .groupby(['data_id']).first().reset_index()

def stop_with_distance_threshold(result_df, filters=None, threshold=0.5):
    if filters is not None:
        result_df = filtering(result_df, [filters])
    tdf = result_df[result_df.min_dist >= threshold]
    return tdf.sort_values("iteration").groupby(['data_id']).first().reset_index()

def stop_at_min_dist_knee(result_df, filters=None, threshold=None):
    """
    increasing curve
    """
    if filters is not None:
        result_df = filtering(result_df, [filters])
    tdf = pd.DataFrame()
    for k, g in result_df.groupby(['data_id', 'method']):
        y = g.min_dist.values
        y[-1] = 1.
        diff = np.diff(np.insert(y, 0, 0.), 1) # first derivative
        if threshold is not None:
            diff[y < threshold] = 0
        knee = diff.argmax()
        tdf = tdf.append(g[g.iteration==knee], ignore_index=True)
    return tdf

def stop_at_hncut_knee(result_df, filters=None):
    """
    decreasing
    """
    if filters is not None:
        result_df = filtering(result_df, [filters])
    result_df['num_clusters'] = result_df['labels_pred'].apply(lambda t: len(set(t)))
    tdf = pd.DataFrame()
    for k, g in result_df.groupby(['data_id', 'method']):
        y = g.hNCut.values/g.num_clusters.values # hncut normalization
        diff = np.insert(y[0:-1], 0, 1.) - y
        tdf = tdf.append(g[g.iteration==diff.argmax()], ignore_index=True)
    return tdf

In [ ]:
def construct_Z(labels_list, distances):
    def sort_clustering_labels(labels):
        map_to_new_index = {}
        for c in labels:
            if c in map_to_new_index:
                continue
            else:
                map_to_new_index[c] = len(map_to_new_index)
        return [map_to_new_index[c] for c in labels]

    labels_list = [sort_clustering_labels(labels) for labels in labels_list]

    num_clusters = None
    for labels in labels_list:
        if num_clusters is None:
            num_clusters = len(set(labels))
        else:
            assert len(set(labels)) == num_clusters - 1
            num_clusters -= 1
    
    cluster_assignment = np.array(range(0, len(labels_list[0])))
    Z = []
    prev_label = np.array(labels_list[0])
    new_cluster_index = len(set(prev_label))
    for i in range(1, len(labels_list)):
        curr_label = np.array(labels_list[i])
        unique_clusters = np.unique(curr_label)
        merged_cluster = None
        for c in unique_clusters:
            child_labels = np.unique(prev_label[curr_label == c])
            assert child_labels.shape[0] in [1, 2]
            if child_labels.shape[0] == 2:
                assert merged_cluster is None
                merged_cluster = c
        children = np.unique(cluster_assignment[curr_label == merged_cluster])
        cluster_assignment[curr_label == merged_cluster] = new_cluster_index
        Z.append([children[0], children[1], round(distances[i-1], 3), (cluster_assignment == new_cluster_index).sum()])
        new_cluster_index += 1
        prev_label = curr_label
    return np.array(Z)

def construct_linkage_matrix(result_df, data_id, affinity, linkage):
    tdf = filtering(result_df, [{'clustering': 'Agglomerative', 'affinity': affinity, 'linkage': linkage, 'data_id': data_id}])
    return construct_Z(tdf.labels_pred.tolist(), tdf.min_dist.tolist())

if Lang == 'Java':
    print(construct_linkage_matrix(result, 'Chart10b-Chart11b-Chart12b', 'hdist', 'average'))

# Drawing Dendrogram

In [ ]:
def plot_dendogram(meta_df, result_df, data_id, affinity, linkage,
                   show_testname=True, color_threshold='auto'):
    # https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/#Inconsistency-Method
    # Annotating dendrogram
    if not np.any(data_id == meta_df.data_id):
        raise Exception(f"No such data: {data_id}")
    colors = ['tab:green', 'tab:orange', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:skyblue']
    fault_names = meta_df[meta_df.data_id == data_id].faults.item()
    ground_truth = meta_df[meta_df.data_id == data_id].labels_true.item()
    test_names = meta_df[meta_df.data_id == data_id].failing_tests.item()
    Z = construct_linkage_matrix(result_df, data_id, affinity, linkage)
    mdist = Z[:,2]

    mdist_diff = np.diff([0] + mdist.tolist() + [1], 1) # first derivative
    mdist_elbow = mdist_diff.argmax()
    final_k = len(mdist_diff) - mdist_elbow

    if color_threshold == 'auto':
        if mdist_elbow == len(mdist):
            color_threshold = (1 + mdist[mdist_elbow-1])/2
        elif mdist_elbow == 0:
            color_threshold = mdist[1]/2
        else:
            color_threshold = (mdist[mdist_elbow] + mdist[mdist_elbow-1])/2
    print(f"K = {final_k}")
    print(f"GT K = {len(fault_names)}")

    plt.figure(
        figsize=(10,min(0.5*len(mdist_diff), 10)))
    plt.title(f'{data_id} ({affinity}-{linkage}): K={final_k}')
    plt.ylabel('Failing Tests')
    plt.xlabel('Distance')

    labels = []
    label_colors = {}
    for i in range(len(test_names)):
        root_cause = fault_names[ground_truth[i]]
        if show_testname:
            testname = test_names[i].split('.')[-1]
            if len(testname) > 12:
                testname = testname[:9]+'...'
            label = f"{testname} ({root_cause})"
        else:
            label = f"t{i+1}\n({root_cause})"
        labels.append(label)
        label_colors[label] = colors[ground_truth[i]]
    dendrogram(
        Z,
        #leaf_font_size=8.,  # font size for the x axis labels
        orientation='left',
        labels=labels,
        color_threshold=color_threshold,
        #link_color_func=lambda k: 'black'
    )
    plt.plot([color_threshold, color_threshold], [0, 10*(Z.shape[0] + 1)], c='red')
    for text in plt.yticks()[1]:
        #print(text.__dict__)
        text._color = label_colors[text._text]
    plt.xlim((1.1, 0))
    plt.grid(True, axis='x')
    #ticks = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
    #plt.xticks(ticks=ticks, labels=[str(round(1-t, 1)) for t in ticks])
    plt.savefig(f'figures/dend/{data_id}_{affinity}_{linkage}.pdf', bbox_inches="tight")
    plt.savefig(f'figures/dend/{data_id}_{affinity}_{linkage}.png', bbox_inches="tight")

    plt.show()

In [ ]:
def plot_objectives(meta_df, result_df, data_id, affinity, linkage,
                    objectives=['modularity', 'hNCut', 'min_dist'],
                    eval_metrics=['normalized_mutual_info_score']):
    tdf = filtering(result_df, 
        [{'clustering': 'Agglomerative', 'affinity': affinity, 'linkage': linkage, 'data_id': data_id}])
    tdf['num_clusters'] = tdf['labels_pred'].apply(lambda t: len(set(t)))
    melted = pd.melt(tdf, id_vars=['num_clusters'],
                     value_vars=objectives + eval_metrics,
                     var_name='Metric', value_name='Value')
    melted['Type'] = melted.Metric.apply(lambda m: 'Objective' if m in objectives else 'Score')
    plt.title(f'{data_id} using {affinity} ({linkage})')
    plt.xticks(range(1, melted.num_clusters.max() + 1))
    sns.lineplot(data=melted, x='num_clusters', y='Value', hue='Metric', style='Type', marker='o')
    plt.legend(bbox_to_anchor=(1.2, 0.35, 0.25, 0.25), loc='center')

    plt.show()

In [ ]:
for data_id in meta[(meta.num_faults >= 3) & (meta.project == 'Closure')].data_id[:100]:
    plot_dendogram(meta, result, data_id, 'hdist', 'average', show_testname=True)
    #plot_dendogram(meta, result, data_id, 'RKT', 'average', show_testname=True)

# Evaluate Stopping Criteria

In [ ]:
def evaluate_stopping_criteria(meta_df, result_df, affinity, linkage,
                               evaluation_metric='normalized_mutual_info_score',
                               project=None, num_faults=None):
    tdf = filtering(result_df, [
        {'clustering': 'Agglomerative', 'affinity': affinity, 'linkage': linkage}])

    tdf = tdf.join(meta_df.set_index('data_id'), on='data_id')
    if project is not None:
        tdf = tdf[tdf.project==project]
    if num_faults is not None:
        tdf = tdf[tdf.num_faults==num_faults]

    N = tdf.data_id.unique().shape[0]

    tdf['num_clusters'] = tdf['labels_pred'].apply(lambda t: len(set(t)))
    tdf['num_clusters/num_faults'] = tdf['num_clusters']/tdf['num_faults']
    tdf['num_clusters==num_faults'] = tdf['num_clusters']==tdf['num_faults']

    if evaluation_metric == 'perfect_ratio':
        tdf['perfect_ratio'] = tdf['normalized_mutual_info_score'] == 1

    eval_result = pd.DataFrame([], columns=['stopping_criterion', f"mean", f"std"])

    # optimal
    optimal_predictions = tdf.sort_values(evaluation_metric, ascending=False).groupby(['data_id']).first()
    assert optimal_predictions.shape[0] == N
    eval_result = eval_result.append({
        'stopping_criterion': 'optimal',
        f'mean': optimal_predictions[evaluation_metric].mean(),
        f'std': optimal_predictions[evaluation_metric].std()
    }, ignore_index=True)

    # optimal
    optimal_predictions = tdf[tdf.num_vertices > 2]\
        .sort_values(evaluation_metric, ascending=False).groupby(['data_id']).first()
    assert optimal_predictions.shape[0] == (meta['num_vertices'] > 2).sum()
    eval_result = eval_result.append({
        'stopping_criterion': 'optimal (>2 failings)',
        f'mean': optimal_predictions[evaluation_metric].mean(),
        f'std': optimal_predictions[evaluation_metric].std()
    }, ignore_index=True)

    # distance threshold
    tick = 0.05
    threshold = 0
    while threshold <= 1:
        predictions = stop_with_distance_threshold(tdf, threshold=threshold)
        assert predictions.shape[0] == N
        eval_result = eval_result.append({
            'stopping_criterion': 'distance_threshold_{:.2f}'.format(threshold),
            'threshold': threshold,
            f'mean': predictions[evaluation_metric].mean(),
            f'std': predictions[evaluation_metric].std()
        }, ignore_index=True)
        threshold += tick

    # maximum modularity
    predictions = stop_at_max_modularity(tdf)
    assert predictions.shape[0] == N
    eval_result = eval_result.append({
        'stopping_criterion': 'max_modularity',
        f'mean': predictions[evaluation_metric].mean(),
        f'std': predictions[evaluation_metric].std()
    }, ignore_index=True)

    # # hncut knee
    # predictions = stop_at_hncut_knee(tdf)
    # assert predictions.shape[0] == N
    # eval_result = eval_result.append({
    #     'stopping_criterion': 'hncut_knee',
    #     f'mean': predictions[evaluation_metric].mean(),
    #     f'std': predictions[evaluation_metric].std()
    # }, ignore_index=True)

    # min_dist knee
    predictions = stop_at_min_dist_knee(tdf)
    assert predictions.shape[0] == N
    eval_result = eval_result.append({
        'stopping_criterion': 'min_dist_knee',
        f'mean': predictions[evaluation_metric].mean(),
        f'std': predictions[evaluation_metric].std()
    }, ignore_index=True)

    # distance threshold + min dist knee
    # tick = 0.05
    # threshold = 0
    # while threshold <= 1:
    #     predictions = stop_at_min_dist_knee(tdf, threshold=threshold)
    #     assert predictions.shape[0] == N
    #     eval_result = eval_result.append({
    #         'stopping_criterion': 'min_dist_knee_after_{:.2f}'.format(threshold),
    #         'threshold': threshold,
    #         f'mean': predictions[evaluation_metric].mean(),
    #         f'std': predictions[evaluation_metric].std()
    #     }, ignore_index=True)
    #     threshold += tick

    if evaluation_metric == 'perfect_ratio':
        return eval_result[['stopping_criterion', 'mean']]

    return eval_result

stop_eval_result = pd.DataFrame()
affinities = ['hdist', 'jaccard', 'dice', 'cosine', 'hamming', 'euclidean', 'RKT']
linkages = ['single', 'average', 'complete']
evaluation_metrics = ['normalized_mutual_info_score', 'perfect_ratio', 'homogeneity_score', 'completeness_score', 'num_clusters/num_faults', 'num_clusters==num_faults']
for affinity, linkage, evaluation_metric in tqdm(list(product(affinities, linkages, evaluation_metrics))):
    eval_result_df = evaluate_stopping_criteria(meta, result, affinity, linkage, evaluation_metric=evaluation_metric)#.sort_values('mean_normalized_mutual_info_score', ascending=False)
    eval_result_df['affinity'] = affinity
    eval_result_df['linkage'] = linkage
    eval_result_df['evaluation_metric'] = evaluation_metric
    stop_eval_result = stop_eval_result.append(eval_result_df, ignore_index=True)

stop_eval_result

# RQ1: The upper bound of clustering performance
The above cell (**evaluate_stopping_criteria**) must have been executed.

## Barchart of Maximum NMI and perfect labelling ratio 

if you want to see the chart
- for all subjects, set `stopping_criterion = 'optimal'` (in the paper)
- for the subjects with > 2 failing test cases, set `stopping_criterion = 'optimal (>2 failings)'`


In [ ]:
target_affinities = ['jaccard', 'dice', 'cosine', 'euclidean', 'hamming', 'RKT', 'hdist']
stopping_criterion = 'optimal' 
# stopping_criterion = 'optimal (>2 failings)'
size = 'large'
if size != 'large':
    plt.figure(figsize=(8, 3))
else:
    # paper size
    plt.figure(figsize=(12, 5))

evaluation_metrics = ['normalized_mutual_info_score', 'perfect_ratio']
print(stop_eval_result[
    (stop_eval_result.stopping_criterion == stopping_criterion)
    & (stop_eval_result.evaluation_metric.isin(evaluation_metrics))
    & (stop_eval_result.affinity.isin(target_affinities))]\
    .pivot(index=['affinity', 'linkage'], columns='evaluation_metric', values='mean')\
    .reset_index()\
    #.sort_values(by='linkage', key=lambda col: [['single', 'average', 'complete'].index(v) for v in col])
    .sort_values(by=['affinity'], key=lambda col: [target_affinities.index(v) for v in col])\
    .round(3).to_markdown(index=False))

for i, evaluation_metric in enumerate(['normalized_mutual_info_score', 'perfect_ratio']):
    plt.subplot(1,2,i+1)
    partial_stop_eval_result = stop_eval_result[
        (stop_eval_result.stopping_criterion == stopping_criterion)
        & (stop_eval_result.evaluation_metric == evaluation_metric)
        & (stop_eval_result.affinity.isin(target_affinities))]
    plt.grid(True, axis='y', zorder=-3)

    sns.barplot(data=partial_stop_eval_result,
        x='affinity', y='mean', hue='linkage', order=target_affinities, zorder=5)
    plt.ylim((max(partial_stop_eval_result['mean'].min() - 0.01, 0), min(partial_stop_eval_result['mean'].max() + 0.025, 1)))
    if 'optimal' in stopping_criterion and evaluation_metric == 'normalized_mutual_info_score':
        plt.title(f"Maximum NMI", size=11)
        plt.ylabel("Average maximum NMI")
        plt.legend(loc="upper left")
    elif 'optimal' in stopping_criterion and evaluation_metric == 'perfect_ratio':
        plt.title(f"Ratio of Perfectly Clustered Subjects", size=11)
        plt.ylabel("Ratio")
        plt.legend(loc="upper left")
    else:
        plt.title(f"{evaluation_metric.replace('_', ' ').capitalize()}\n{stopping_criterion} ({Lang} subjects)")
    #plt.xlabel("Distance Metric")
    plt.xlabel(None)
    plt.xticks(rotation=50, size=11)
plt.subplots_adjust(wspace=0.3)
if size == 'large':
    plt.savefig(f"figures/eval/{Lang}_{stopping_criterion}_large.pdf", bbox_inches='tight')
    plt.savefig(f"figures/eval/{Lang}_{stopping_criterion}_large.png", bbox_inches='tight')
else:
    plt.savefig(f"figures/eval/{Lang}_{stopping_criterion}.pdf", bbox_inches='tight')
    plt.savefig(f"figures/eval/{Lang}_{stopping_criterion}.png", bbox_inches='tight')

plt.show()

## RQ1 supplementary result: maximum NMI histogram

In [ ]:
def max_NMI_frame(meta_df, result_df, filters_list):
    multi_faults = meta_df[(meta_df['num_vertices'] > 2)]['data_id']

    sdf = filtering(result_df, filters_list)
    mdf = sdf[sdf.data_id.isin(multi_faults)]

    max_nmi_score = mdf.groupby(['data_id', 'method']).max()['normalized_mutual_info_score'].to_frame()
    max_nmi_score["maximum NMI"] = max_nmi_score["normalized_mutual_info_score"]
    return max_nmi_score.reset_index()

def max_NMI_analysis(meta_df, result_df, filters_list):
    max_nmi_score = max_NMI_frame(meta_df, result_df, filters_list)
    
    # T-Test
    groups = max_nmi_score.groupby("method")
    for i, g1 in enumerate(groups):
        method1, df1 = g1
        for j, g2 in enumerate(groups):
            method2, df2 = g2
            if not i < j:
                continue
            """
            Calculate the t-test on TWO RELATED samples of scores, a and b.
            """
            x, y = df1["maximum NMI"].values, df2["maximum NMI"].values
            t_stat, p_value = stats.ttest_rel(x,y)
            print(p_value)
            if p_value < 0.05:
                #print(f"Reject the null hypothesis of identical average scores: {method1}, {method2}")
                if x.mean() < y.mean():
                    print(f"{method1} < {method2} (pvalue={round(p_value, 3)})")
                else:
                    print(f"{method2} < {method1} (pvalue={round(p_value, 3)})")
            else:
                print(f"{method1} ~ {method2} (accept null hypothesis)")
    # Histogram
    plt.figure()
    plt.title("Histogram of Maximum NMI scores")

    sns.histplot(data=max_nmi_score,
                 x="maximum NMI", binwidth=0.1,
                 hue="method", multiple="dodge", kde=True,
                 bins=np.arange(0, 1.1, 0.1))

    plt.show()

if Lang == 'Java':
    max_NMI_analysis(meta, result, filters_list=[
        {'clustering': 'Agglomerative','affinity': 'hdist', 'linkage': 'average'},
        {'clustering': 'Agglomerative','affinity': 'RKT', 'linkage': 'single'}
    ])
elif Lang == 'C':
    max_NMI_analysis(meta, result, filters_list=[
        {'clustering': 'Agglomerative','affinity': 'hdist', 'linkage': 'single'},
        {'clustering': 'Agglomerative','affinity': 'euclidean', 'linkage': 'average'}
    ])

# RQ2: Evaluating the result with a stopping criterion for agglomerative clustering

## Evaluation of baseline techniques: MSeer and TCN (in the paper)

In [ ]:
if Lang == 'Java':
    baselines = ['MSeer', 'TCN']
else:
    baselines = ['MSeer']

result["perfect_ratio"] = result["normalized_mutual_info_score"].apply(lambda s: s == 1.0)

result['num_clusters'] = result['labels_pred'].apply(lambda t: len(set(t)))
result['num_clusters/num_faults'] = result['num_clusters']/result.join(meta.set_index('data_id'), on='data_id')['num_faults'].astype('float32')
result['num_clusters==num_faults'] = (result['num_clusters']==result.join(meta.set_index('data_id'), on='data_id')['num_faults'])

print(result[result.clustering.isin(baselines)].groupby(['clustering']).mean()\
    [['num_clusters/num_faults', 'num_clusters==num_faults', 'homogeneity_score', 'completeness_score', 'normalized_mutual_info_score', 'perfect_ratio']].round(3).reset_index().to_markdown(index=False))
result[result.clustering.isin(baselines)].groupby(['clustering']).mean()\
    [['num_clusters/num_faults', 'num_clusters==num_faults', 'homogeneity_score', 'completeness_score', 'normalized_mutual_info_score', 'perfect_ratio']].round(3)

## Using Algorithm 2: Elbow point of mdist curve (in the paper)

In [ ]:
tdf = stop_eval_result[
    (stop_eval_result.stopping_criterion == 'min_dist_knee')
    & (stop_eval_result.affinity != 'euclidean') # since it's unnormalised
]
evaluation_metrics = ['num_clusters/num_faults', 'num_clusters==num_faults', 'homogeneity_score', 'completeness_score', 'normalized_mutual_info_score', 'perfect_ratio']
data = {}
for m in evaluation_metrics:
    mdf = tdf[stop_eval_result.evaluation_metric == m][['affinity', 'linkage', 'mean']].round(3)
    data[m] = mdf.set_index(['affinity', 'linkage'])['mean'].rename(m)
print(pd.concat(data, axis=1).reset_index().to_markdown(index=False))

## Using max-modularity stopping criterion

In [ ]:
tdf = stop_eval_result[
    (stop_eval_result.stopping_criterion == 'max_modularity')
]
evaluation_metrics = ['num_clusters/num_faults', 'num_clusters==num_faults', 'homogeneity_score', 'completeness_score', 'normalized_mutual_info_score', 'perfect_ratio']
data = {}
for m in evaluation_metrics:
    mdf = tdf[stop_eval_result.evaluation_metric == m][['affinity', 'linkage', 'mean']].round(3)
    data[m] = mdf.set_index(['affinity', 'linkage'])['mean'].rename(m)
print(pd.concat(data, axis=1).reset_index().to_markdown(index=False))

## Averaged NMI scores for each distance threshold

In [ ]:
evaluation_metric = 'normalized_mutual_info_score' # 'perfect_ratio'
plt.figure(figsize=(12, 10))
stop_eval_result['is_dist_threshold'] = stop_eval_result['stopping_criterion']\
    .apply(lambda s: s.startswith("distance_threshold_"))
sns.lineplot(data=stop_eval_result[
        stop_eval_result.is_dist_threshold
        & (stop_eval_result.evaluation_metric == evaluation_metric)
        & (stop_eval_result.affinity != 'euclidean') # since it's unnormalised
    ], x='threshold', y='mean', hue='affinity', style='linkage')
plt.title("NMI for each distance threshod")
plt.grid(True, axis='y')
if Lang == 'Java':
    plt.ylim((0.35, 0.9))

else:
    plt.ylim((0.20, 0.8))
plt.ylabel("Averaged NMI")
plt.savefig(f"figures/eval/NMI_distance_threshold_{Lang}.pdf", bbox_inches='tight')
plt.savefig(f"figures/eval/NMI_distance_threshold_{Lang}.png", bbox_inches='tight')
plt.show()

stop_eval_result[
        stop_eval_result.is_dist_threshold
        & (stop_eval_result.evaluation_metric == evaluation_metric)
        & (stop_eval_result.affinity != 'euclidean') # since it's unnormalised
].sort_values(['mean'], ascending=False)

## Drawing Comparison Graphs

In [ ]:
stopping_to_method = {
   'Agglomerative-max-modularity': stop_at_max_modularity,
   'Agglomerative-elbow-method': stop_at_min_dist_knee
}
result = result[~result.clustering.isin(stopping_to_method.keys())]

for key, df in result[result['clustering'] == 'Agglomerative'].groupby(['affinity', 'linkage']):
   affinity, linkage = key
   print(f"Processing: {key}")
   for criterion in stopping_to_method:
      if affinity == 'euclidean' and criterion != 'Agglomerative-max-modularity':
         continue
      df = filtering(result, [{'clustering': criterion, 'affinity': affinity, 'linkage': linkage}])
      df = stopping_to_method[criterion](result,
         {'clustering': 'Agglomerative', 'affinity': affinity, 'linkage': linkage})
      df['clustering'] = criterion
      df['min_dist'] = float("inf")
      result = result.append(df, ignore_index=True)

In [ ]:
def compare_stopping_criteria(meta_df, result_df, affinity, linkage,
                   eval_metric='normalized_mutual_info_score',
                   distance_thresholds=np.arange(0, 1.01, 0.05),
                   savepath=None,
                   show_errorband=True):
    if affinity != 'euclidean':
        filters_list=[
            {'clustering': 'Agglomerative-elbow-method', 'affinity': affinity, 'linkage': linkage},
            {'clustering': 'Agglomerative', 'affinity': affinity, 'linkage': linkage},
            {'clustering': 'Agglomerative-max-modularity', 'affinity': affinity, 'linkage': linkage},
        ]
    else:
        filters_list=[
            {'clustering': 'Agglomerative-max-modularity', 'affinity': affinity, 'linkage': linkage},
        ]
    merged_frame = pd.DataFrame()

    hue_order = []
    for filters in filters_list:
        for threshold in distance_thresholds:
            pred = stop_with_distance_threshold(result_df, filters, threshold)
            pred['threshold'] = threshold
            merged_frame = merged_frame.append(pred)
        hue_order.append(filters['clustering'])

    merged_frame['perfect'] = merged_frame['normalized_mutual_info_score'] == 1
    if not show_errorband or eval_metric == 'perfect':
        merged_frame = merged_frame.groupby(['threshold', 'clustering']).mean()[eval_metric]
        merged_frame = merged_frame.reset_index()

    plt.title(f"{affinity}-{linkage}")
    sns.lineplot(data=merged_frame, x='threshold', y=eval_metric,
        hue='clustering', hue_order=hue_order, ci=68, legend=(linkage == 'complete'))
    if linkage != 'single':
        plt.ylabel(None)

affinities = ['jaccard', 'dice', 'cosine', 'euclidean', 'hamming', 'RKT', 'hdist']
linkages = ['single', 'average', 'complete']
for affinity in affinities:
    plt.figure(figsize=(11, 3))
    for i, linkage in enumerate(linkages):
        plt.subplot(1, 3, i+1)
        compare_stopping_criteria(
            meta, result, eval_metric='normalized_mutual_info_score',
            affinity=affinity, linkage=linkage, show_errorband=False)
    plt.legend(bbox_to_anchor=(1.5, 0.35, 0.25, 0.25), loc='center')
    plt.savefig(f"figures/eval/stop_{affinity}_{Lang}.png", bbox_inches='tight')
    plt.show()

# \[RQ3\] Analysis of Distance Calculation Cost

Run this cell only once, and draw the **distance calculation cost** graph (right below this cell) for both `Java` and `C`. Then results of both languages will be save in `merged_cost_df`. With the dataframe, you can draw the **RKT complexity analysis** graph.

In [ ]:
merged_cost_df = pd.DataFrame()

## Distance calculation cost (in the paper)

In [ ]:
target_dist = ['jaccard', 'dice', 'cosine', 'hamming', 'RKT', 'hdist']
covered_reduction_needed_dist = set(target_dist) - set({'RKT', 'hdist'})

joined_cost_df = dc_result[dc_result.affinity.isin(target_dist)].set_index(['data_id', 'affinity']).join(meta.set_index(['data_id']), on='data_id').reset_index()

update_index = joined_cost_df.affinity == 'hdist'
joined_cost_df.loc[update_index, 'cost'] = joined_cost_df[update_index].cost + joined_cost_df[update_index].hypergraph_modeling_cost

update_index = joined_cost_df.affinity.isin(covered_reduction_needed_dist)
joined_cost_df.loc[update_index, 'cost'] = joined_cost_df[update_index].cost + joined_cost_df[update_index].coverage_reduction_cost

# joined_cost_df['log10(sec)'] = np.log10(joined_cost_df['cost'])

joined_cost_df['Language'] = Lang
if merged_cost_df.shape[0] == 0 or np.all(merged_cost_df['Language'] != Lang):
    merged_cost_df = merged_cost_df.append(joined_cost_df, ignore_index=True)

#plt.figure(figsize=(6, 2.5))
plt.figure(figsize=(10, 4))
plt.title(f"Log-scaled distance calculation cost (s)")

major_logS = range(-4, 6, 2)
minor_logS = range(-4, 5, 1)

major_ticks = [10**n for n in major_logS]
minor_ticks = [10**n for n in minor_logS]

for i in range(len(major_ticks) - 1):
    bins = 11
    gap = major_ticks[i+1] - major_ticks[i]
    width = gap/bins
    ticks = [round(major_ticks[i]+width*j, 5) for j in range(0, bins)]
    for tick in ticks:
        plt.plot([tick, tick], [-0.5, 6.5], c='#ddd', zorder=-1, linestyle='-')

for tick in [n for n in major_ticks]:
    plt.plot([tick, tick], [-0.5, 6.5], c='#aaa', zorder=-1)

# for n in logS:
#     """
#     guide lines
#     """
#     plt.plot([n, n], [-0.5, 6.5], c='lightgrey', zorder=-1)

# 1. stripplot
# sns.stripplot(data=merged_cost_df, y='affinity', x='log10(sec)', order=target_dist,
#               hue='Language', dodge=True, alpha=.25, zorder=3)

# 2. boxplot
plt.xscale('log')
#plt.grid(True, axis='x', which='both')
ax = sns.boxplot(data=merged_cost_df, y='affinity', x='cost', order=target_dist, hue='Language', zorder=100)
plt.xticks([n for n in minor_ticks], ["1e{}s".format(n) if abs(n) > 2 else (str(10**n) + "s") for n in minor_logS])

plt.ylabel('Distance Metric')


plt.xlabel('')
plt.savefig(f"figures/eval/distance_cost_boxplot.pdf", bbox_inches="tight")
plt.savefig(f"figures/eval/distance_cost_boxplot.png", bbox_inches="tight")

plt.show()

In [ ]:
(merged_cost_df[merged_cost_df.affinity == 'hdist'].cost < 1).all()

## RKT complexity analysis (in the paper)

In [ ]:
RKT_cost = merged_cost_df[merged_cost_df.affinity=='RKT']
RKT_cost['squared M'] = np.power(RKT_cost['num_total_components'], 2)
RKT_cost['squared |T_F|'] = np.power(RKT_cost['num_vertices'], 2)
RKT_cost['cost_estimator'] = RKT_cost['squared M'] * RKT_cost['squared |T_F|']
RKT_cost = RKT_cost.sort_values(['project'])

plt.figure(figsize=(4,2.5))
#plt.figure(figsize=(10, 5))

sns.scatterplot(data=RKT_cost, x='cost_estimator', y='cost', hue='project', style='project', s=60)
plt.title(f'Time complexity analysis of RKT')
plt.xlabel(r'$M^2 \times |T_F|^2$')
plt.ylabel('RKT calculation time (s)')
slope, intercept, r_value, p_value, std_err = stats.linregress(x=RKT_cost.cost_estimator.astype('float32'),
                                                               y=RKT_cost.cost.astype('float32'))
print("r_squared", r_value ** 2)
print(r_value, p_value, std_err)
plt.plot(RKT_cost.cost_estimator.astype('float32'),
         slope * RKT_cost.cost_estimator.astype('float32') + intercept,
         zorder=-1, color='lightgrey', linestyle='dashed')
plt.text(3.2 * 1e12, 6000, "y = {:.2g}x {:.2g}\n(R-squared: {:.4f})".format(slope, intercept, r_value**2), color='#111')
plt.legend(bbox_to_anchor=(1.05, 0.38, 0.25, 0.25), loc='center')
#plt.legend(bbox_to_anchor=(1.10, 0.35, 0.25, 0.25), loc='center')

plt.grid(False)

plt.savefig(f"figures/eval/RKT_time_complexity.pdf", bbox_inches='tight')
plt.savefig(f"figures/eval/RKT_time_complexity.png", bbox_inches='tight')


In [ ]:
from sklearn.linear_model import LinearRegression
hdist_cost = merged_cost_df[merged_cost_df.affinity=='hdist']
hdist_cost['squared |T_F|'] = np.power(hdist_cost['num_vertices'], 2)
X = hdist_cost[['num_hyperedges', 'num_total_components', 'squared |T_F|']]
y = hdist_cost['cost']

size = 'small'
linreg = LinearRegression()
linreg.fit(X, y)
a, b, c = linreg.coef_

hdist_cost['cost_estimator'] = a * hdist_cost['num_hyperedges'] + b * hdist_cost['num_total_components'] + c * hdist_cost['squared |T_F|']

hdist_cost = hdist_cost.sort_values(['project'])
if size == 'large':
    plt.figure(figsize=(10, 5))
else:
    plt.figure(figsize=(7,2))
#plt.figure(figsize=(10, 5))

sns.scatterplot(data=hdist_cost, x='cost_estimator', y='cost', hue='project', style='project', s=60)

plt.title(f'Cost analysis of hdist')

plt.ylabel('hdist calculation time (s)')
slope, intercept, r_value, p_value, std_err = stats.linregress(x=hdist_cost.cost_estimator.astype('float32'),
                                                               y=hdist_cost.cost.astype('float32'))
print("r_squared", r_value ** 2)
print(r_value, p_value, std_err)
plt.plot(hdist_cost.cost_estimator.astype('float32'),
         slope * hdist_cost.cost_estimator.astype('float32') + intercept,
         zorder=-1, color='lightgrey', linestyle='dashed')
        

if size == 'large':
    plt.legend(bbox_to_anchor=(0.95, 0.35, 0.25, 0.25), loc='center')
    plt.xlabel(r"$({:.3g})M + ({:.3g})E + ({:.3g})|T_F|^2$".format(b, c, a))
    plt.ylabel('hdist calculation time (s)')
    plt.text(0.5, 0.2, "y = {:.3g}x {:.2g}\n(R-squared: {:.4f})".format(slope, intercept, r_value**2), color='#111')
else:
    plt.legend(bbox_to_anchor=(1.00, 0.35, 0.25, 0.25), loc='center')
    plt.xlabel(r"$x = 10^{}({:.2f}M' + {:.2f}M + {:.2f}|T_F|^2)$".format("{-5}", a*10**5, b*10**5, c*10**5))
    plt.ylabel('time (s)')
    plt.text(0.4, 0.0, "time (s) = x {:.2g}\n(R-squared: {:.4f})".format(intercept, r_value**2), color='#111')

# plt.grid(False)
if size == 'large':
    plt.savefig(f"figures/eval/hdist_time_complexity_large.pdf", bbox_inches='tight')
    plt.savefig(f"figures/eval/hdist_time_complexity_large.png", bbox_inches='tight')
else:
    plt.savefig(f"figures/eval/hdist_time_complexity.pdf", bbox_inches='tight')
    plt.savefig(f"figures/eval/hdist_time_complexity.png", bbox_inches='tight')


In [ ]:
joined_cost_df[joined_cost_df.data_id == 'Closure49b-Closure50b']

# \[RQ4\] Evaluate FL results

In [ ]:
def draw_figure(fl_eval_df, savepath=None):
    melted = pd.DataFrame(columns=['clustering', 'metric', 'n', 'value'])
    for i, row in fl_eval_df.iterrows():
        clustering = row['clustering']
        melted = melted.append({'clustering': clustering, 'metric': 'R.R.', 'value': row['R.R.']}, ignore_index=True)
        for n in N:
            melted = melted.append({'clustering': clustering, 'metric': 'found_faults', 'n': n, 'value': row[f'found_faults (n={n})']}, ignore_index=True)
            melted = melted.append({'clustering': clustering, 'metric': 't_wef', 'n': n, 'value': row[f't_wef (n={n})']}, ignore_index=True)

    plt.figure(figsize=(12, 10))

    minor_ticks = [x+0.5 for x in range(melted.clustering.nunique())]

    plt.subplot(3, 1, 1)
    plt.title("Found Faults (higher is better)")
    ax = sns.barplot(data=melted[melted.metric == 'found_faults'], x='clustering', y='value', hue='n')
    ax.set_xticks([], minor=False)
    ax.set_xticks(minor_ticks, minor=True)
    ax.grid(which='minor', axis='x')
    plt.xlabel(None)
    plt.legend(bbox_to_anchor=(0.93, 0.35, 0.25, 0.25), loc='center', title='n')

    plt.subplot(3, 1, 2)
    plt.title("t-wef (lower is better)")
    ax = sns.barplot(data=melted[melted.metric == 't_wef'], x='clustering', y='value', hue='n')
    ax.set_xticks([], minor=False)
    ax.set_xticks(minor_ticks, minor=True)
    ax.grid(which='minor', axis='x')
    plt.xlabel(None)
    plt.legend(bbox_to_anchor=(0.93, 0.35, 0.25, 0.25), loc='center', title='n')

    plt.subplot(3, 1, 3)
    plt.title("R.R. (lower is better)")
    melted.clustering.nunique()
    ax = sns.barplot(data=melted[melted.metric == 'R.R.'], x='clustering', y='value')
    plt.xticks(rotation=90)
    ax.set_xticks(minor_ticks, minor=True)
    plt.grid(True, which='minor')

    if savepath:
        plt.savefig(savepath, bbox_inches='tight')
    plt.show()

def evaluate_FL(meta_df, result_df, fl_result_df,
    filters=None,
    formula='ochiai',
    tie_breaker='max',
    stopping_criterion=stop_at_min_dist_knee,
    top_N=10):

    has_faulty_components = meta['data_id'][meta['faulty_components'].apply(lambda l: len(l) > 0)]

    fl_result_df = fl_result_df[(fl_result_df.formula == formula) & (fl_result_df.tie_breaker == tie_breaker)]

    if filters is None:
        fl_result_df['num_clusters'] = fl_result_df['labels_pred'].apply(lambda t: len(set(t)))
        prediction_ranks = fl_result_df[fl_result_df.num_clusters == 1].set_index(['data_id', 'faulty_component_index'])[['cluster', 'rank']].astype('int32')
        method = 'No Clustering'
        assert prediction_ranks['rank'].isnull().sum() == 0
    else:
        tdf = filtering(result_df, [filters])
        N = tdf.data_id.unique().shape[0]
        if filters['clustering'] == 'Agglomerative':
            predictions = stopping_criterion(tdf)
        else:
            predictions = tdf
        assert predictions.shape[0] == N
        
        prediction_ranks = predictions.join(fl_result_df.set_index(['data_id', 'labels_pred']), on=['data_id', 'labels_pred'])
        method = prediction_ranks['method'].values[0]
        prediction_ranks = prediction_ranks[prediction_ranks.data_id.isin(has_faulty_components)]
        #print(prediction_ranks[prediction_ranks['rank'].isnull()])
        assert prediction_ranks['rank'].isnull().sum() == 0
        num_ranks = prediction_ranks.groupby(['data_id', 'faulty_component_index']).count()['cluster'].reset_index()
        for k, g in num_ranks.groupby(['data_id']):
            arr = g.cluster.values
            # Check if all items in an array are equal
            assert np.max(arr) == np.min(arr)

        prediction_ranks = prediction_ranks.sort_values(['data_id', 'faulty_component_index'])\
            .set_index(['data_id', 'faulty_component_index'])[['cluster', 'rank']]#.astype('int32')

    root_cause_df = pd.DataFrame(columns=['data_id', 'fault_index', 'faulty_component_index'])
    for index, row in meta_df.iterrows():
        for fault_index, faulty_components_list in enumerate(row['root_cause_map']):
            for faulty_component_index in faulty_components_list:
                root_cause_df = root_cause_df.append({
                    'data_id': row['data_id'],
                    'fault_index': fault_index,
                    'faulty_component_index': faulty_component_index
                }, ignore_index=True)

    prediction_ranks = root_cause_df.join(prediction_ranks, on=['data_id', 'faulty_component_index'])
    num_total_faults = prediction_ranks.groupby(['data_id']).fault_index.nunique()
    num_total_faults = num_total_faults.rename("num_total_faults")
    prediction_ranks = prediction_ranks[~prediction_ranks['rank'].isnull()]


    first_fixed = prediction_ranks.groupby(['data_id', 'cluster'])['rank'].min().to_frame()\
        .reset_index().set_index(['data_id', 'cluster', 'rank']).join(
            prediction_ranks.set_index(['data_id', 'cluster', 'rank']),
            how='left'
        ).reset_index(level=['rank'])

    num_rankings_per_data = prediction_ranks.groupby('data_id').cluster.nunique()

    topmost = prediction_ranks.sort_values('rank').groupby(['data_id', 'faulty_component_index']).first()

    row = {
        'clustering': method if 'Agglomerative' not in method else method.replace('Agglomerative', 'AHC'),
        'R.R.': 1 - (topmost.groupby('data_id').cluster.nunique()/num_rankings_per_data).mean()
    }

    for n in top_N + ['inf']:
        if n != 'inf':
            num_first_fixed = first_fixed[first_fixed['rank'] <= n].groupby(['data_id']).fault_index.nunique().rename("num_fixed_faults")
        else:
            num_first_fixed = first_fixed.groupby(['data_id']).fault_index.nunique().rename("num_fixed_faults")
        joined = pd.concat([num_total_faults, num_first_fixed], axis=1, join='outer')
        joined.loc[joined.num_fixed_faults.isnull(), 'num_fixed_faults'] = 0
        if n != 'inf':
            first_fixed[f'wef_{n}'] = first_fixed['rank'].apply(lambda r: r - 1 if r <= n else n)
        else:
            first_fixed[f'wef_{n}'] = first_fixed['rank'] - 1
        row[f"found_faults (n={n})"] = (joined['num_fixed_faults']/joined['num_total_faults']).mean()
        row[f"t_wef (n={n})"] = first_fixed.groupby(['data_id', 'cluster'])\
                                .min()[f'wef_{n}'].groupby(['data_id']).sum().mean()

    return row


baselines = [
    None, {'clustering': 'MSeer'}
]
if Lang == 'Java':
    N=[1, 5, 10, 15]
    baselines.append({'clustering': 'TCN'})
else:
    N=[1, 5, 10, 15]

for formula in ['ochiai', 'crosstab']:
    for tie_breaker in ['min', 'max']:
        print(formula, tie_breaker)
        fl_eval_df = pd.DataFrame(
            columns=['clustering', 'R.R.'] + sum([[f"found_faults (n={n})", f"t_wef (n={n})"] for n in N+['inf']], []))

        print("Baselines")
        for baseline in tqdm(baselines):
            row = evaluate_FL(meta, result, fl_result, formula=formula, tie_breaker=tie_breaker, top_N=N, filters=baseline)
            fl_eval_df = fl_eval_df.append(row, ignore_index=True)

        affinities = ['jaccard', 'dice', 'cosine', 'hamming', 'RKT', 'hdist']
        linkages = ['single', 'average', 'complete']
        for affinity, linkage in tqdm(list(product(affinities, linkages))):
            row = evaluate_FL(
                meta, result, fl_result, formula=formula, tie_breaker=tie_breaker, top_N=N,
                filters={'clustering': 'Agglomerative', 'affinity': affinity, 'linkage': linkage},
                stopping_criterion=stop_at_min_dist_knee
            )
            fl_eval_df = fl_eval_df.append(row, ignore_index=True)
        print(fl_eval_df.round(3).to_markdown(index=False))
        #print(fl_eval_df.round(2).to_latex(index=False))
        draw_figure(fl_eval_df, savepath=f"figures/eval/FL_{formula}_{tie_breaker}_{Lang}.png")